In [3]:
import os

def load_data(path):
    # Load input file
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data.split("\n")

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import GRU, Dense, TimeDistributed, Dropout

import numpy as np
# Build the RNN layers
def simple_model(input_shape, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    model = Sequential()
    model.add(GRU(128, input_shape=input_shape[1:], return_sequences=True))
    model.add(Dropout(0.5))
    model.add(GRU(128, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(256, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size + 1, activation='softmax')))

    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])
    return model

# Load English data
english_sentences = load_data('./small_vocab_en')
# Load French data
french_sentences = load_data('./small_vocab_fr')

french_tokenizer = Tokenizer()
english_tokenizer = Tokenizer()

In [4]:
# preprocessing - tokenize and pad
# write a tokenize function
def tokenize(tokenizer, x):
    """
    Tokenize x
    :param tokenizer: Tokenizer object
    :param x: List of sentences/strings to be tokenized
    :return: tokenized x data
    """
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x)


# preprocess sentences in x and y = tokenize + pad + reshape
def preprocess(x, x_tokenizer, y, y_tokenizer):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param x_tokenzier: tokenizer for x
    :param y: Label List of sentences
    :param: y_tokenizer: tokenizer for y
    :return: Tuple of (Preprocessed x, Preprocessed y)
    """
    raw_x = tokenize(x_tokenizer, x)
    raw_y = tokenize(y_tokenizer, y)

    raw_x = pad_sequences(raw_x, padding='post')
    raw_y = pad_sequences(raw_y, padding='post')

    max_y_len = raw_y.shape[1]
    max_x_len = raw_x.shape[1]
    if raw_y.shape[1] > raw_x.shape[1]:
        preprocess_x = pad_sequences(raw_x, maxlen=max_y_len, padding='post')
        preprocess_y = raw_y
    else:
        if max_y_len == max_x_len:
            preprocess_y = raw_y
        else:
            preprocess_y = pad_sequences(raw_y, maxlen=max_x_len, padding='post')
        preprocess_x = raw_x
    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    preprocess_x = preprocess_x.reshape(*preprocess_x.shape, 1)

    return preprocess_x, preprocess_y


preproc_english_sentences, preproc_french_sentences = preprocess(english_sentences,
                                                                 english_tokenizer,
                                                                 french_sentences,
                                                                 french_tokenizer)
french_vocab_size = len(french_tokenizer.word_index)

# Train the neural network
model = simple_model(
    preproc_english_sentences.shape,
    french_vocab_size)
model.summary()

history = model.fit(preproc_english_sentences, preproc_french_sentences, batch_size=300,
                     epochs=2, validation_split=0.2)

def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    # SOLUTION PART 2: DO NOT PROVIDE TO STUDENT
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = "<PAD>"

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

2023-04-03 17:53:36.398894: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-03 17:53:36.399332: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 21, 128)           50304     
                                                                 
 dropout (Dropout)           (None, 21, 128)           0         
                                                                 
 gru_1 (GRU)                 (None, 21, 128)           99072     


2023-04-03 17:53:36.652994: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-03 17:53:36.655085: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-03 17:53:36.656236: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

                                                                 
 dropout_1 (Dropout)         (None, 21, 128)           0         
                                                                 
 time_distributed (TimeDistr  (None, 21, 256)          33024     
 ibuted)                                                         
                                                                 
 dropout_2 (Dropout)         (None, 21, 256)           0         
                                                                 
 time_distributed_1 (TimeDis  (None, 21, 345)          88665     
 tributed)                                                       
                                                                 
Total params: 271,065
Trainable params: 271,065
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2


2023-04-03 17:53:37.121871: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-03 17:53:37.123995: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-03 17:53:37.125142: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

368/368 [==============================] - ETA: 0s - loss: 2.2283 - accuracy: 0.5067

2023-04-03 17:54:21.202432: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-03 17:54:21.204031: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-03 17:54:21.205415: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

368/368 [==============================] - 48s 124ms/step - loss: 2.2283 - accuracy: 0.5067 - val_loss: 1.4947 - val_accuracy: 0.5942
Epoch 2/2
368/368 [==============================] - 45s 123ms/step - loss: 1.5123 - accuracy: 0.5892 - val_loss: 1.2816 - val_accuracy: 0.6239


In [5]:
print("English sentence:", english_sentences[0])
print("Actual translation:", french_sentences[0])
print("Predicted translation:", logits_to_text(model.predict(preproc_english_sentences[:1])[0], french_tokenizer))

English sentence: new jersey is sometimes quiet during autumn , and it is snowy in april .
Actual translation: new jersey est parfois calme pendant l' automne , et il est neigeux en avril .


2023-04-03 17:55:22.226312: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-03 17:55:22.227723: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-03 17:55:22.228686: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 474ms/step
Predicted translation: new jersey est parfois chaud en mois et il est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [6]:
from keras.models import save_model
save_model(model, "translation.tf")

2023-04-03 17:55:41.116163: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-03 17:55:41.117464: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-03 17:55:41.118442: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: translation.tf/assets


INFO:tensorflow:Assets written to: translation.tf/assets
